In [ ]:
import sqlite3
import json

# Connect to the SQLite database
conn = sqlite3.connect('iotaDB.db')
cursor = conn.cursor()

# Query the database for the JSON strings of addresses and amounts
cursor.execute("SELECT input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y FROM Transactions")
rows = cursor.fetchall()

# Dictionary to hold the net amounts for each address (outputs - inputs)
address_balances = {}

# Iterate over each row to parse JSON and accumulate amounts
for input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y in rows:
    # Parse the JSON data
    input_addresses = json.loads(input_addresses_x)
    input_amounts = json.loads(input_amounts_x)
    output_addresses = json.loads(output_addresses_y)
    output_amounts = json.loads(output_amounts_y)
    
    # Subtract input amounts from the corresponding addresses
    for address, amount in zip(input_addresses, input_amounts):
        if amount != 0:
            if address in address_balances:
                address_balances[address] -= amount
            else:
                address_balances[address] = -amount

    # Add output amounts to the corresponding addresses
    for address, amount in zip(output_addresses, output_amounts):
        if amount != 0:
            if address in address_balances:
                address_balances[address] += amount
            else:
                address_balances[address] = amount

# Close the database connection
conn.close()

# Optionally, print the balance for each address
for address, balance in address_balances.items():
    print(f"Address: {address}, Net Balance: {balance}")


In [ ]:
import sqlite3
import json
import pandas as pd
import matplotlib.pyplot as plt

# Connect to the SQLite database
conn = sqlite3.connect('iotaDB.db')
cursor = conn.cursor()

# Query the database for the JSON strings of addresses and amounts
cursor.execute("SELECT input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y FROM Transactions")
rows = cursor.fetchall()

# Dictionary to hold the net amounts for each address (outputs - inputs)
address_balances = {}

# Iterate over each row to parse JSON and accumulate amounts
for input_addresses_x, input_amounts_x, output_addresses_y, output_amounts_y in rows:
    # Parse the JSON data
    input_addresses = json.loads(input_addresses_x)
    input_amounts = json.loads(input_amounts_x)
    output_addresses = json.loads(output_addresses_y)
    output_amounts = json.loads(output_amounts_y)
    
    # Subtract input amounts from the corresponding addresses
    for address, amount in zip(input_addresses, input_amounts):
        if amount != 0:
            if address in address_balances:
                address_balances[address] -= amount
            else:
                address_balances[address] = -amount

    # Add output amounts to the corresponding addresses
    for address, amount in zip(output_addresses, output_amounts):
        if amount != 0:
            if address in address_balances:
                address_balances[address] += amount
            else:
                address_balances[address] = amount

# Close the database connection
conn.close()

# Create a DataFrame from the address balances
df = pd.DataFrame(list(address_balances.items()), columns=['Address', 'Net Balance'])

# Sort by 'Net Balance' in descending order and pick the top ten
top_ten_df = df.sort_values(by='Net Balance', ascending=False).head(10)

# Plotting
plt.figure(figsize=(12, 8))
plt.barh(top_ten_df['Address'], top_ten_df['Net Balance'], color='skyblue')
plt.xlabel('Net Balance')
plt.ylabel('Address')
plt.title('Top 10 Addresses by Net Balance')
plt.gca().invert_yaxis()  # Invert y-axis to have the highest values on top
plt.show()
